In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.io import impute_missing_values
from pepbench.datasets import EmpkinsDataset, GuardianDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../../")

In [3]:
result_path = root_path.joinpath("results")
result_path

PosixPath('../../../results')

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

/Users/sebas/Development/ResearchInternship/Data/2024_08_PEP_Benchmarking/EmpkinS_Dataset


#### Set flags for further processing

In [5]:
save_results = True
include_rr_interval = True
include_b_point_reference = False
drop_nan = True
impute_nan = False

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")

Sampling rate ICG: 1000
Sampling rate ICG: 500


In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

#### Load data from the challenge results

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_b_point").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
).per_sample
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_b_point").resolve(), index_cols_per_sample=["participant", "phase"]
).per_sample

In [9]:
results_empkins = results_empkins.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])
results_empkins = results_empkins.drop(columns=['level_6'])
results_empkins

/tmp/ipykernel_3956807/1857737345.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  results_empkins = results_empkins.drop(columns=['level_6'])


heartbeat_id  \
                                                                                                  estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                  
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep               0   
                                                                                       Prep               1   
                                                                                       Prep               2   
                                                                                       Prep               3   
                                                                                       Prep               4   
...                                                                                                     ...   
vanlien2013-42-ms b-point-reference none                         VP_032      ftsst     Pause_5            7   
                                                                                       Pause_5            8   
                                                                                       Pause_5            9   
                                                                                       Pause_5           10   
                                                                                       Pause_5           11   

                                                                                                          \
                                                                                               reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase               
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep            0   
                                                                                       Prep            1   
                                                                                       Prep            2   
                                                                                       Prep            3   
                                                                                       Prep            4   
...                                                                                                  ...   
vanlien2013-42-ms b-point-reference none                         VP_032      ftsst     Pause_5         7   
                                                                                       Pause_5         8   
                                                                                       Pause_5         9   
                                                                                       Pause_5        10   
                                                                                       Pause_5        11   

                                                                                               heartbeat_start_sample  \
                                                                                                            estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                            
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                       394   
                                                                                       Prep                       984   
                                                                                       Prep                      1569   
                                                                                       Prep                      2134   
                                                                                       Prep                      2684   
...                                                        

#### Introduce column 'condition' to the guardian dataset to assure compatibility with the empkins dataset

In [10]:
results_guardian = results_guardian.assign(condition="Dummy")
results_guardian = results_guardian.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase']).drop(columns=['level_5'])
results_guardian

/tmp/ipykernel_3956807/1754320087.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  results_guardian = results_guardian.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase']).drop(columns=['level_5'])


heartbeat_id  \
                                                                                                   estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                   
forounzafar2018   b-point-reference none                         GDN0005     Dummy     Pause               0   
                                                                                       Pause               1   
                                                                                       Pause               2   
                                                                                       Pause               3   
                                                                                       Pause               4   
...                                                                                                      ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown           51   
                                                                                       TiltDown           52   
                                                                                       TiltDown           53   
                                                                                       TiltDown           54   
                                                                                       TiltDown           55   

                                                                                                           \
                                                                                                reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                
forounzafar2018   b-point-reference none                         GDN0005     Dummy     Pause            0   
                                                                                       Pause            1   
                                                                                       Pause            2   
                                                                                       Pause            3   
                                                                                       Pause            4   
...                                                                                                   ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown        51   
                                                                                       TiltDown        52   
                                                                                       TiltDown        53   
                                                                                       TiltDown        54   
                                                                                       TiltDown        55   

                                                                                                heartbeat_start_sample  \
                                                                                                             estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                             
forounzafar2018   b-point-reference none                         GDN0005     Dummy     Pause                       352   
                                                                                       Pause                       808   
                                                                                       Pause                      1252   
                                                                                       Pause                      1686   
                                                                                       Pause                      2116   
...                     

#### Perform data preprocessing

In [11]:
results_empkins_norm = results_empkins.copy()
results_guardian_norm = results_guardian.copy()
results_empkins_norm[("q_wave_onset_sample", "estimated")] = results_empkins[("q_wave_onset_sample", "estimated")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("q_wave_onset_sample", "reference")] = results_empkins[("q_wave_onset_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("b_point_sample", "reference")] = results_empkins[("b_point_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_wave_onset_sample", "estimated")] = results_guardian[("q_wave_onset_sample", "estimated")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_wave_onset_sample", "reference")] = results_guardian[("q_wave_onset_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "reference")] = results_guardian[("b_point_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]

In [12]:
cols = [("heartbeat_start_sample", "estimated"), ("heartbeat_start_sample", "reference"), ("heartbeat_end_sample", "estimated"), ("heartbeat_end_sample", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("pep_sample", "estimated"), ("pep_sample", "reference")]
results_empkins_cleaned_ms = results_empkins_norm.copy()
results_guardian_cleaned_ms = results_guardian_norm.copy()
results_empkins_cleaned_ms[cols] = results_empkins_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_empkins)
results_guardian_cleaned_ms[cols] = results_guardian_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_guardian)

#### Concatenate the empkins and guardian dataframes

In [13]:
preprocessed_results_empkins_guardian = pd.concat([results_empkins_cleaned_ms, results_guardian_cleaned_ms])
preprocessed_results_empkins_guardian

heartbeat_id  \
                                                                                                   estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                   
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                0   
                                                                                       Prep                1   
                                                                                       Prep                2   
                                                                                       Prep                3   
                                                                                       Prep                4   
...                                                                                                      ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown           51   
                                                                                       TiltDown           52   
                                                                                       TiltDown           53   
                                                                                       TiltDown           54   
                                                                                       TiltDown           55   

                                                                                                           \
                                                                                                reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep             0   
                                                                                       Prep             1   
                                                                                       Prep             2   
                                                                                       Prep             3   
                                                                                       Prep             4   
...                                                                                                   ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown        51   
                                                                                       TiltDown        52   
                                                                                       TiltDown        53   
                                                                                       TiltDown        54   
                                                                                       TiltDown        55   

                                                                                                heartbeat_start_sample  \
                                                                                                             estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                             
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                      394.0   
                                                                                       Prep                      984.0   
                                                                                       Prep                     1569.0   
                                                                                       Prep                     2134.0   
                                                                                       Prep                     2684.0   
...                     

In [14]:
input_data = preprocessed_results_empkins_guardian.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])

#### Select columns

In [15]:
if include_rr_interval:
    input_data = input_data[[("heartbeat_id", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("rr_interval_ms", "estimated")]]
elif include_b_point_reference:
    input_data = input_data[[("heartbeat_id", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("rr_interval_ms", "estimated"), ("b_point_sample", "reference")]]
else:
    input_data = input_data[[("heartbeat_id", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference")]]
input_data.columns = input_data.columns.to_flat_index().str.join("_")
input_data

heartbeat_id_reference  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                              
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                           0   
                                                                                       Prep                           1   
                                                                                       Prep                           2   
                                                                                       Prep                           3   
                                                                                       Prep                           4   
...                                                                                                                 ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown                      51   
                                                                                       TiltDown                      52   
                                                                                       TiltDown                      53   
                                                                                       TiltDown                      54   
                                                                                       TiltDown                      55   

                                                                                                 q_wave_onset_sample_estimated  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                                     
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                              187.0   
                                                                                       Prep                              186.0   
                                                                                       Prep                              182.0   
                                                                                       Prep                              174.0   
                                                                                       Prep                              170.0   
...                                                                                                                        ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown                          338.0   
                                                                                       TiltDown                          336.0   
                                                                                       TiltDown                          346.0   
                                                                                       TiltDown                          324.0   
                                                                                       TiltDown                          316.0   

                                                                                                 q_wave_onset_sample_reference  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                                     
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                              173.0   
                                                                                       Prep                              172.0   
                                                                                       Prep                              166.0   
                                                                                       Prep                              157.0   
                          

In [16]:
print(f"Amount of algorithms: {len(input_data.index.get_level_values('q_wave_algorithm').unique())}")

Amount of algorithms: 9


#### Transform dataframe from long to wide format

In [17]:
input_data = input_data.reset_index()
# Pivot the DataFrame
if include_rr_interval:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference', 'rr_interval_ms_estimated'],
        columns="q_wave_algorithm",
        values='q_wave_onset_sample_estimated'
    ).reset_index()
elif include_b_point_reference:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference', 'b_point_sample_reference'],
        columns="q_wave_algorithm",
        values='q_wave_onset_sample_estimated'
    ).reset_index()
else:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference'],
        columns="q_wave_algorithm",
        values='q_wave_onset_sample_estimated'
    ).reset_index()

input_data_wide

q_wave_algorithm,participant,condition,phase,heartbeat_id_reference,q_wave_onset_sample_reference,rr_interval_ms_estimated,forounzafar2018,martinez2004,scipy-findpeaks,vanlien2013-32-ms,vanlien2013-34-ms,vanlien2013-36-ms,vanlien2013-38-ms,vanlien2013-40-ms,vanlien2013-42-ms
0,GDN0005,Dummy,HoldingBreath,0,260.0,850.0,276.0,262.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
1,GDN0005,Dummy,HoldingBreath,1,260.0,778.0,278.0,262.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
2,GDN0005,Dummy,HoldingBreath,3,222.0,746.0,236.0,224.0,224.0,230.0,228.0,226.0,224.0,222.0,220.0
3,GDN0005,Dummy,HoldingBreath,4,220.0,766.0,236.0,222.0,222.0,230.0,228.0,226.0,224.0,222.0,220.0
4,GDN0005,Dummy,HoldingBreath,5,228.0,790.0,242.0,230.0,230.0,236.0,234.0,232.0,230.0,228.0,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,172.0,619.0,183.0,173.0,173.0,177.0,175.0,173.0,171.0,169.0,167.0
11607,VP_032,tsst,Talk,40,181.0,680.0,193.0,182.0,182.0,185.0,183.0,181.0,179.0,177.0,175.0
11608,VP_032,tsst,Talk,41,200.0,719.0,210.0,201.0,201.0,206.0,204.0,202.0,200.0,198.0,196.0
11609,VP_032,tsst,Talk,42,212.0,698.0,223.0,213.0,213.0,220.0,218.0,216.0,214.0,212.0,210.0


#### Drop nan values and the 'scipy-findpeaks' column

In [18]:
print(f"Drop nan values: {drop_nan}")
print(f"Impute nan values: {impute_nan}")

Drop nan values: True
Impute nan values: False


#### Delete rows where the column 'rr_interval_ms_estimated' contains nan values

In [19]:
if include_rr_interval:
    input_data_wide = input_data_wide.dropna(subset=['rr_interval_ms_estimated'])
input_data_wide

q_wave_algorithm,participant,condition,phase,heartbeat_id_reference,q_wave_onset_sample_reference,rr_interval_ms_estimated,forounzafar2018,martinez2004,scipy-findpeaks,vanlien2013-32-ms,vanlien2013-34-ms,vanlien2013-36-ms,vanlien2013-38-ms,vanlien2013-40-ms,vanlien2013-42-ms
0,GDN0005,Dummy,HoldingBreath,0,260.0,850.0,276.0,262.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
1,GDN0005,Dummy,HoldingBreath,1,260.0,778.0,278.0,262.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
2,GDN0005,Dummy,HoldingBreath,3,222.0,746.0,236.0,224.0,224.0,230.0,228.0,226.0,224.0,222.0,220.0
3,GDN0005,Dummy,HoldingBreath,4,220.0,766.0,236.0,222.0,222.0,230.0,228.0,226.0,224.0,222.0,220.0
4,GDN0005,Dummy,HoldingBreath,5,228.0,790.0,242.0,230.0,230.0,236.0,234.0,232.0,230.0,228.0,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,172.0,619.0,183.0,173.0,173.0,177.0,175.0,173.0,171.0,169.0,167.0
11607,VP_032,tsst,Talk,40,181.0,680.0,193.0,182.0,182.0,185.0,183.0,181.0,179.0,177.0,175.0
11608,VP_032,tsst,Talk,41,200.0,719.0,210.0,201.0,201.0,206.0,204.0,202.0,200.0,198.0,196.0
11609,VP_032,tsst,Talk,42,212.0,698.0,223.0,213.0,213.0,220.0,218.0,216.0,214.0,212.0,210.0


In [20]:
input_data_wide_cleaned = input_data_wide.drop(columns=["scipy-findpeaks"])
if include_rr_interval:
    input_data_wide_cleaned = input_data_wide_cleaned.reset_index().set_index(['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference', 'rr_interval_ms_estimated'], append=True)
else:
    input_data_wide_cleaned = input_data_wide_cleaned.reset_index().set_index(['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference'], append=True)
input_data_wide_cleaned.index

MultiIndex([(    0, 'GDN0005', 'Dummy', 'HoldingBreath',  0, 260.0, 850.0),
            (    1, 'GDN0005', 'Dummy', 'HoldingBreath',  1, 260.0, 778.0),
            (    2, 'GDN0005', 'Dummy', 'HoldingBreath',  3, 222.0, 746.0),
            (    3, 'GDN0005', 'Dummy', 'HoldingBreath',  4, 220.0, 766.0),
            (    4, 'GDN0005', 'Dummy', 'HoldingBreath',  5, 228.0, 790.0),
            (    5, 'GDN0005', 'Dummy', 'HoldingBreath',  6, 236.0, 768.0),
            (    6, 'GDN0005', 'Dummy', 'HoldingBreath',  7, 226.0, 826.0),
            (    7, 'GDN0005', 'Dummy', 'HoldingBreath',  9, 258.0, 844.0),
            (    8, 'GDN0005', 'Dummy', 'HoldingBreath', 10, 256.0, 802.0),
            (    9, 'GDN0005', 'Dummy', 'HoldingBreath', 11, 242.0, 770.0),
            ...
            (11495,  'VP_032',  'tsst',          'Talk', 34, 186.0, 633.0),
            (11496,  'VP_032',  'tsst',          'Talk', 35, 181.0, 631.0),
            (11497,  'VP_032',  'tsst',          'Talk', 36, 183.0, 604.

In [21]:
validation_index = input_data_wide_cleaned[input_data_wide_cleaned.isna().any(axis=1)].index
validation_index

MultiIndex([(   64, 'GDN0005', 'Dummy', 'HoldingBreath', 67, 252.0,  862.0),
            (  126, 'GDN0005', 'Dummy',         'Pause', 63, 314.0,  948.0),
            (  322, 'GDN0006', 'Dummy',         'Pause',  4, 342.0,  448.0),
            (  371, 'GDN0006', 'Dummy',         'Pause', 54, 338.0, 1048.0),
            (  426, 'GDN0006', 'Dummy',      'TiltDown', 54, 344.0, 1088.0),
            (  490, 'GDN0006', 'Dummy',        'TiltUp', 63, 302.0,  942.0),
            (  782, 'GDN0007', 'Dummy',        'TiltUp', 69, 252.0,  654.0),
            (  839, 'GDN0007', 'Dummy',      'Valsalva', 57, 426.0, 1154.0),
            (  960, 'GDN0008', 'Dummy',         'Pause', 55, 330.0, 1096.0),
            ( 1012, 'GDN0008', 'Dummy',      'TiltDown', 51, 352.0, 1184.0),
            ...
            (11127,  'VP_031',  'tsst',       'Pause_5', 16, 145.0,  509.0),
            (11240,  'VP_031',  'tsst',          'Talk', 60, 139.0,  487.0),
            (11280,  'VP_032', 'ftsst',          'Math', 39,

In [22]:
if drop_nan:
    input_data_wide_cleaned = input_data_wide_cleaned.dropna()
else:
    if impute_nan:
        input_data_wide_cleaned = input_data_wide_cleaned.drop(columns=['index'])
        input_data_wide_cleaned = impute_missing_values(input_data_wide_cleaned, mode='median')
        print(f"Amount of rows after imputation: {input_data_wide_cleaned.shape[0]}")
        input_data_wide_cleaned = input_data_wide_cleaned.dropna()
        print(f"Amount of rows after deleting all nan rows: {input_data_wide_cleaned.shape[0]}")       
input_data_wide_cleaned

,,,,,,q_wave_algorithm,index,forounzafar2018,martinez2004,vanlien2013-32-ms,vanlien2013-34-ms,vanlien2013-36-ms,vanlien2013-38-ms,vanlien2013-40-ms,vanlien2013-42-ms
,participant,condition,phase,heartbeat_id_reference,q_wave_onset_sample_reference,rr_interval_ms_estimated,,,,,,,,,
0,GDN0005,Dummy,HoldingBreath,0,260.0,850.0,0,276.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
1,GDN0005,Dummy,HoldingBreath,1,260.0,778.0,1,278.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
2,GDN0005,Dummy,HoldingBreath,3,222.0,746.0,2,236.0,224.0,230.0,228.0,226.0,224.0,222.0,220.0
3,GDN0005,Dummy,HoldingBreath,4,220.0,766.0,3,236.0,222.0,230.0,228.0,226.0,224.0,222.0,220.0
4,GDN0005,Dummy,HoldingBreath,5,228.0,790.0,4,242.0,230.0,236.0,234.0,232.0,230.0,228.0,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11499,VP_032,tsst,Talk,38,153.0,596.0,11605,188.0,154.0,179.0,177.0,175.0,173.0,171.0,169.0
11500,VP_032,tsst,Talk,39,172.0,619.0,11606,183.0,173.0,177.0,175.0,173.0,171.0,169.0,167.0
11501,VP_032,tsst,Talk,40,181.0,680.0,11607,193.0,182.0,185.0,183.0,181.0,179.0,177.0,175.0


In [24]:
input_data_wide_cleaned.loc[validation_index]

KeyError: "None of [MultiIndex([(   64, 'GDN0005', 'Dummy', 'HoldingBreath', 67, 252.0,  862.0),\n            (  126, 'GDN0005', 'Dummy',         'Pause', 63, 314.0,  948.0),\n            (  322, 'GDN0006', 'Dummy',         'Pause',  4, 342.0,  448.0),\n            (  371, 'GDN0006', 'Dummy',         'Pause', 54, 338.0, 1048.0),\n            (  426, 'GDN0006', 'Dummy',      'TiltDown', 54, 344.0, 1088.0),\n            (  490, 'GDN0006', 'Dummy',        'TiltUp', 63, 302.0,  942.0),\n            (  782, 'GDN0007', 'Dummy',        'TiltUp', 69, 252.0,  654.0),\n            (  839, 'GDN0007', 'Dummy',      'Valsalva', 57, 426.0, 1154.0),\n            (  960, 'GDN0008', 'Dummy',         'Pause', 55, 330.0, 1096.0),\n            ( 1012, 'GDN0008', 'Dummy',      'TiltDown', 51, 352.0, 1184.0),\n            ...\n            (11127,  'VP_031',  'tsst',       'Pause_5', 16, 145.0,  509.0),\n            (11240,  'VP_031',  'tsst',          'Talk', 60, 139.0,  487.0),\n            (11280,  'VP_03

#### Save the preprocessed data to use it for training

In [25]:
print(f"Save results: {save_results}")
print(f"Include RR-Interval: {include_rr_interval}")
print(f"Drop missing values: {drop_nan}")
print(f"Impute missing values: {impute_nan}")
print(f"Include B-Point reference: {include_b_point_reference}")

Save results: True
Include RR-Interval: True
Drop missing values: True
Impute missing values: False
Include B-Point reference: False


In [26]:
if save_results:
    input_data_wide_cleaned.to_csv(result_path.joinpath("data/q-peak/rr-interval/train_data_q_peak_rr_interval.csv"))
    print("Data saved!")

Data saved!


In [27]:
test_data = pd.read_csv(result_path.joinpath("data/q-peak/rr-interval/train_data_q_peak_rr_interval.csv"), index_col=[0,1,2,3,4,5])
test_data

,,,,,,rr_interval_ms_estimated,index,forounzafar2018,martinez2004,vanlien2013-32-ms,vanlien2013-34-ms,vanlien2013-36-ms,vanlien2013-38-ms,vanlien2013-40-ms,vanlien2013-42-ms
,participant,condition,phase,heartbeat_id_reference,q_wave_onset_sample_reference,,,,,,,,,,
0,GDN0005,Dummy,HoldingBreath,0,260.0,850.0,0,276.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
1,GDN0005,Dummy,HoldingBreath,1,260.0,778.0,1,278.0,262.0,266.0,264.0,262.0,260.0,258.0,256.0
2,GDN0005,Dummy,HoldingBreath,3,222.0,746.0,2,236.0,224.0,230.0,228.0,226.0,224.0,222.0,220.0
3,GDN0005,Dummy,HoldingBreath,4,220.0,766.0,3,236.0,222.0,230.0,228.0,226.0,224.0,222.0,220.0
4,GDN0005,Dummy,HoldingBreath,5,228.0,790.0,4,242.0,230.0,236.0,234.0,232.0,230.0,228.0,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11499,VP_032,tsst,Talk,38,153.0,596.0,11605,188.0,154.0,179.0,177.0,175.0,173.0,171.0,169.0
11500,VP_032,tsst,Talk,39,172.0,619.0,11606,183.0,173.0,177.0,175.0,173.0,171.0,169.0,167.0
11501,VP_032,tsst,Talk,40,181.0,680.0,11607,193.0,182.0,185.0,183.0,181.0,179.0,177.0,175.0


In [28]:
test_data.isna().any()

rr_interval_ms_estimated    False
index                       False
forounzafar2018             False
martinez2004                False
vanlien2013-32-ms           False
vanlien2013-34-ms           False
vanlien2013-36-ms           False
vanlien2013-38-ms           False
vanlien2013-40-ms           False
vanlien2013-42-ms           False
dtype: bool